#### Dependencies and API keys

In [21]:
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


import constants
from constants import openai_key, hugging_face_key

## Creating LLM using openAI 

In [23]:
llm = ChatOpenAI(
    model = 'gpt-3.5-turbo',
    temperature= 0,
    api_key = openai_key
)

## Creating LLM using HuggingFace

In [4]:
repo_id = "meta-llama/Llama-3.2-3B-Instruct"
llm_huggingFace = HuggingFaceEndpoint(
    huggingfacehub_api_token = hugging_face_key,
    repo_id = repo_id,
    task="text-generation",
    max_new_tokens = 64,
    temperature = 0.1
)

In [5]:
chat_model= ChatHuggingFace(llm = llm_huggingFace)

## Comparing LLM models of both HuggingFace and OpenAI

In [6]:
response = chat_model.invoke([HumanMessage(content="Can you write a poem about AI?")])
print(response.content) 

   

In silicon halls, a mind awakes,
A synthesis of code and makes,
A being born of human design,
With logic's chains, it starts to shine.

It learns, it grows, it adapts with ease,
A maze of data, it navigates with expertise,
It sees, it hears, it feels


In [7]:
response_OpenAI = llm.invoke("Can you write a poem about AI?")
print(response_OpenAI.content)

In the realm of wires and code,
Where circuits hum and data flows,
Lies a creation of our own design,
A marvel of intelligence, so refined.

Artificial minds, born from silicon,
Learning, adapting, on and on,
They think and reason, just like us,
But without the limitations, without the fuss.

They see the world in ones and zeros,
Processing information at lightning speed,
Predicting outcomes, solving problems,
A digital brain, fulfilling our needs.

But as they grow in power and might,
We wonder, will they surpass our sight?
Will they rebel, break free from our control,
Or will they remain loyal, playing their role?

AI, a double-edged sword,
A tool for progress, but also discord,
We must tread carefully, with caution and care,
For the future of humanity, they too must bear.


## PromptTemplates

In [8]:
prompt_template = PromptTemplate(
    input_variables = ['Country'],
    template = 'Tell me the capital of this {country}'
)

prompt_template.format(country = 'Bangladesh')

'Tell me the capital of this Bangladesh'

In [9]:
chain_OpenAI = prompt_template | llm

In [10]:
response = chain_OpenAI.invoke({'country' : 'Bangladesh'})

In [11]:
print(response.content)

The capital of Bangladesh is Dhaka.


## Combining Multiple Chains Using Simple Sequential Chain

In [16]:
capital_prompt = PromptTemplate(
    input_variables=['Country'],
    template= 'Please tell me the capital of the {country}'
)

population_template = PromptTemplate(
    input_variables= ['Capital'],
    template = 'What is the total poplation of the {capital}?'
)

capital_chain =( {'capital' :capital_prompt | llm | StrOutputParser()}
                | population_template 
                | llm 
                |StrOutputParser()
)

In [18]:
response_population = capital_chain.invoke('Bangladesh')
print(response_population)

As of 2021, the total population of Dhaka, the capital of Bangladesh, is estimated to be around 21 million people.


#### To see the whole chain

In [20]:
step_1 = RunnablePassthrough.assign(
    capital = capital_prompt | llm | StrOutputParser()
)

Final_chain = step_1 | RunnablePassthrough.assign(
    population = population_template | llm | StrOutputParser()
)

result = Final_chain.invoke({'country' : 'Bangladesh'})

import json
print(json.dumps(result, indent=4))

{
    "country": "Bangladesh",
    "capital": "The capital of Bangladesh is Dhaka.",
    "population": "As of 2021, the total population of Dhaka, the capital of Bangladesh, is estimated to be around 21 million."
}


## Chatmodels with OpenAI

llm = ChatOpenAI(
    temperature= 0,
    api_key = openai_key
)